In [ ]:
import requests
import pandas as pd
import plotly.express as px

liste_villes = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg",
"Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon",
"Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer",
"Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]


COORDONNEES D'UNE SEULE VILLE

In [40]:
# ESSAI MANUEL POUR UNE SEULE VILLE 

url = "https://nominatim.openstreetmap.org/search?"
headers = {"accept": "application/json", 'user_agent':'charles.mittempergher@gmail.com'}
p = {'city' : 'Toulouse', 'country':'France', 'format':'json', 'limit':1}

response = requests.get(url, headers=headers, params=p)

BOUCLE POUR LES COORDONNEES TOUTES LES VILLES

In [41]:
# RECUPERATION DES DONNEES POUR TOUTE LA LISTE DES VILLES
url = "https://nominatim.openstreetmap.org/search?"
headers = {'user_agent':'charles.mittempergher@gmail.com'}

df_villes = pd.DataFrame(columns=['ville','lat','lon'])

for i in range(len(liste_villes)):
    
    p = {'city' : {liste_villes[i]}, 'country':'France', 'format':'json', 'limit':1, 'email':'charles@gmail.com'}

    r = requests.get(url, headers=headers, params=p)
    ville = r.json()[0]['name']
    lat   = r.json()[0]['lat']
    lon   = r.json()[0]['lon']

    df_villes.loc[i] = [ville, lat,lon]
    print(f'Ajout de {liste_villes[i]} effectué')

Ajout de Mont Saint Michel effectué
Ajout de St Malo effectué
Ajout de Bayeux effectué
Ajout de Le Havre effectué
Ajout de Rouen effectué
Ajout de Paris effectué
Ajout de Amiens effectué
Ajout de Lille effectué
Ajout de Strasbourg effectué
Ajout de Chateau du Haut Koenigsbourg effectué
Ajout de Colmar effectué
Ajout de Eguisheim effectué
Ajout de Besancon effectué
Ajout de Dijon effectué
Ajout de Annecy effectué
Ajout de Grenoble effectué
Ajout de Lyon effectué
Ajout de Gorges du Verdon effectué
Ajout de Bormes les Mimosas effectué
Ajout de Cassis effectué
Ajout de Marseille effectué
Ajout de Aix en Provence effectué
Ajout de Avignon effectué
Ajout de Uzes effectué
Ajout de Nimes effectué
Ajout de Aigues Mortes effectué
Ajout de Saintes Maries de la mer effectué
Ajout de Collioure effectué
Ajout de Carcassonne effectué
Ajout de Ariege effectué
Ajout de Toulouse effectué
Ajout de Montauban effectué
Ajout de Biarritz effectué
Ajout de Bayonne effectué
Ajout de La Rochelle effectué


In [42]:
r.content

b'[{"place_id":266833066,"licence":"Data \xc2\xa9 OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright","osm_type":"relation","osm_id":117858,"lat":"46.159732","lon":"-1.1515951","class":"boundary","type":"administrative","place_rank":16,"importance":0.6409735277294301,"addresstype":"town","name":"La Rochelle","display_name":"La Rochelle, Charente-Maritime, Nouvelle-Aquitaine, France m\xc3\xa9tropolitaine, 17000, France","boundingbox":["46.1331804","46.1908971","-1.2419231","-1.1110970"]}]'

In [43]:
df_villes.head()

,ville,lat,lon
0,Mont Saint-Michel,48.6359541,-1.511459954959514
1,Saint-Malo,48.649518,-2.0260409
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658


 DEBUT DE LA PARTIE AVEC OPENWEATHER

In [44]:
url_ow = "https://api.openweathermap.org/data/2.5/forecast?"

lat = 43.2961743
lon = 5.3699525
p = {'lat' : lat, 'lon' : lon, 'units':'metric', 'appid' : 'a2fa8b199ebbdb821bb767d8318aa7bd'}
response = requests.get(url_ow, params=p)


In [45]:
response.json().get('list')

[{'dt': 1731607200,
  'main': {'temp': 12.29,
   'feels_like': 10.98,
   'temp_min': 12.29,
   'temp_max': 14.79,
   'pressure': 1022,
   'sea_level': 1022,
   'grnd_level': 1014,
   'humidity': 54,
   'temp_kf': -2.5},
  'weather': [{'id': 800,
    'main': 'Clear',
    'description': 'clear sky',
    'icon': '01n'}],
  'clouds': {'all': 0},
  'wind': {'speed': 2.34, 'deg': 318, 'gust': 2.45},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'n'},
  'dt_txt': '2024-11-14 18:00:00'},
 {'dt': 1731618000,
  'main': {'temp': 12.83,
   'feels_like': 11.6,
   'temp_min': 12.83,
   'temp_max': 13.92,
   'pressure': 1022,
   'sea_level': 1022,
   'grnd_level': 1015,
   'humidity': 55,
   'temp_kf': -1.09},
  'weather': [{'id': 800,
    'main': 'Clear',
    'description': 'clear sky',
    'icon': '01n'}],
  'clouds': {'all': 2},
  'wind': {'speed': 0.83, 'deg': 305, 'gust': 0.86},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'n'},
  'dt_txt': '2024-11-14 21:00:00'},
 {'dt': 1731628800

CREATION DU DATAFRAME AVEC LES PREVISIONS METEO POUR UNE SEULE VILLE

In [46]:
liste_previsions_une_ville = response.json().get('list')

len(liste_previsions_une_ville)
df_prevision_une_ville = pd.DataFrame(columns=['Ville','date','heure', 'temp','id_type_temps','type_temps'])
for i in range(len(liste_previsions_une_ville)):
    
    ville = 'Marseille'
    dt_text= liste_previsions_une_ville[i]['dt_txt']
    date = dt_text.split(" ")[0]
    heure= dt_text.split(" ")[1]
    temp=liste_previsions_une_ville[i]['main']['temp']
    id_type_temps=liste_previsions_une_ville[i]['weather'][0]['id']
    type_temps=liste_previsions_une_ville[i]['weather'][0]['main']

    df_prevision_une_ville.loc[i] = [ville,date,heure,temp,id_type_temps,type_temps]

In [47]:
df_prevision_une_ville.head()

,Ville,date,heure,temp,id_type_temps,type_temps
0,Marseille,2024-11-14,18:00:00,12.29,800,Clear
1,Marseille,2024-11-14,21:00:00,12.83,800,Clear
2,Marseille,2024-11-15,00:00:00,13.10,800,Clear
3,Marseille,2024-11-15,03:00:00,13.00,800,Clear
4,Marseille,2024-11-15,06:00:00,12.73,800,Clear


ON PREND PAS LES PREVISIONS ENTRE MINUIT ET 6H

In [48]:
mask = df_prevision_une_ville['heure'] >= '06:00:00'
data = df_prevision_une_ville[mask]

In [49]:
data.head()

,Ville,date,heure,temp,id_type_temps,type_temps
0,Marseille,2024-11-14,18:00:00,12.29,800,Clear
1,Marseille,2024-11-14,21:00:00,12.83,800,Clear
4,Marseille,2024-11-15,06:00:00,12.73,800,Clear
5,Marseille,2024-11-15,09:00:00,13.82,800,Clear
6,Marseille,2024-11-15,12:00:00,14.84,800,Clear


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

PASSAGE AUX CHOSES SERIEUSES

BOUCLE POUR TOUTES LES PREVISIONS DE TOUTES LES VILLES

In [50]:
url_ow = "https://api.openweathermap.org/data/2.5/forecast?"

df_prevision_toutes_les_villes = pd.DataFrame(columns=['Ville','date','heure','Part_of_day', 'temp','id_type_temps','type_temps','couv_nuageuse','vent','prob_pluie','pluie'])

# BOUCLE SUR LES VILLES 
for v in range(len(df_villes)):

    lat = df_villes.loc[v,"lat"]
    lon = df_villes.loc[v,"lon"]
    p = {'lat' : lat, 'lon' : lon, 'units':'metric', 'appid' : 'a2fa8b199ebbdb821bb767d8318aa7bd'}
    response = requests.get(url_ow, params=p)
    response.json().get('list')

    liste_previsions_une_ville_final = response.json().get('list')
    nb_prev = len(liste_previsions_une_ville_final)
    # BOUCLES SUR LES PREVISIONS
    for i in range(len(liste_previsions_une_ville_final)):
        
        ville = df_villes.loc[v,"ville"]
        dt_text= liste_previsions_une_ville_final[i]['dt_txt']
        date = dt_text.split(" ")[0]
        heure= dt_text.split(" ")[1]
        temp=liste_previsions_une_ville_final[i]['main']['temp']
        id_type_temps=liste_previsions_une_ville_final[i]['weather'][0]['id']
        type_temps=liste_previsions_une_ville_final[i]['weather'][0]['main']
        couv_nuageuse=liste_previsions_une_ville_final[i]['clouds']['all']
        vent=liste_previsions_une_ville_final[i]['wind']['speed']
        prob_pluie=liste_previsions_une_ville_final[i]['pop']
        pluie = 0
        if 'rain' in liste_previsions_une_ville_final[i].keys():
            pluie = liste_previsions_une_ville_final[i]['rain']['3h']
        pod = liste_previsions_une_ville_final[i]['sys']['pod']

        df_prevision_toutes_les_villes.loc[len(df_prevision_toutes_les_villes)+1] = [ville,date,heure,pod,temp,id_type_temps,type_temps,couv_nuageuse,vent,prob_pluie,pluie]
        

In [51]:
mask = df_prevision_toutes_les_villes['heure'] >= '06:00:00'
data_toutes_les_villes = df_prevision_toutes_les_villes[mask]

ON REGARDE LES DIFFERENTES VALEURS DU TYPE DE METEO POUR ETABLIR UN CLASSEMENT

In [52]:
data_toutes_les_villes['id_type_temps'].unique()

array([801, 800, 803, 802, 804, 500, 501, 601], dtype=int64)

MAPPING ET CREATION D'UNE NOUVELLE COLONNE AVEC LA NOTE DE LA PREVISION

In [53]:
classement = {800: 10, 801:9,802:8,803:7,804:6,500:4,501:3, 601:2}
data_toutes_les_villes['note_type_temps']=data_toutes_les_villes['id_type_temps'].apply(lambda x: classement[x] if x in classement else 1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_19596\3510448894.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [54]:
data_toutes_les_villes.head()

,Ville,date,heure,Part_of_day,temp,id_type_temps,type_temps,couv_nuageuse,vent,prob_pluie,pluie,note_type_temps
1,Mont Saint-Michel,2024-11-14,18:00:00,n,11.29,801,Clouds,14,3.10,0.0,0.0,9
2,Mont Saint-Michel,2024-11-14,21:00:00,n,10.15,800,Clear,10,2.78,0.0,0.0,10
5,Mont Saint-Michel,2024-11-15,06:00:00,n,7.17,801,Clouds,14,1.94,0.0,0.0,9
6,Mont Saint-Michel,2024-11-15,09:00:00,d,9.18,803,Clouds,79,3.25,0.0,0.0,7
7,Mont Saint-Michel,2024-11-15,12:00:00,d,11.53,802,Clouds,50,2.47,0.0,0.0,8


In [55]:
total_par_ville = data_toutes_les_villes.groupby('Ville')['note_type_temps'].sum()

In [56]:
total_par_ville

Ville
Aigues-Mortes                  231
Aix-en-Provence                239
Amiens                         176
Annecy                         228
Avignon                        239
Bayeux                         183
Bayonne                        194
Besançon                       215
Biarritz                       191
Bormes-les-Mimosas             244
Carcassonne                    230
Cassis                         236
Château du Haut-Kœnigsbourg    196
Collioure                      230
Colmar                         200
Dijon                          206
Eguisheim                      201
Gorges du Verdon               247
Grenoble                       235
La Rochelle                    207
Le Havre                       164
Lille                          173
Lyon                           225
Marseille                      236
Mont Saint-Michel              187
Montauban                      223
Nîmes                          239
Paris                          191
Rouen         

In [57]:
moyenne_temp_par_ville = data_toutes_les_villes.groupby('Ville')['temp'].mean()

ON FUSIONNE LES TROIS TABLES POUR AVOIR NOS DONNEES FINALES (VILLES ET COORDONNEES, SCORE, TEMPERATURE MOYENNE)

In [58]:
df_final = df_villes.merge(total_par_ville,left_on='ville',right_on='Ville')
df_final = df_final.merge(moyenne_temp_par_ville,left_on='ville', right_on='Ville')

CHANGEMENT DU TYPE DE DONNEES POUR LATITUDE ET LONGITUDE

In [59]:
df_final['lat'] = df_final['lat'].astype('float64')
df_final['lon'] = df_final['lon'].astype('float64')

AFFICHAGE DU DATASET FINAL UTILISE POUR LA CARTE

In [60]:
df_final.sort_values("note_type_temps", ascending=False)

,ville,lat,lon,note_type_temps,temp
17,Gorges du Verdon,43.749656,6.328562,247,6.569667
18,Bormes-les-Mimosas,43.150697,6.341928,244,13.256333
24,Nîmes,43.837425,4.360069,239,11.295667
21,Aix-en-Provence,43.529842,5.447474,239,12.158667
22,Avignon,43.949249,4.805901,239,10.872000
19,Cassis,43.214036,5.539632,236,13.787667
20,Marseille,43.296174,5.369953,236,14.491333
15,Grenoble,45.187560,5.735782,235,9.318333
23,Uzès,44.012128,4.419672,231,10.389333
25,Aigues-Mortes,43.566152,4.191540,231,13.034333


CREATION DE LA CARTE

In [61]:
fig = px.scatter_mapbox(df_final,lat="lat", lon="lon", color="note_type_temps", hover_name="ville", hover_data='temp', height=700,
mapbox_style="open-street-map",title='Où on part ce weekend ?',zoom=4.5,color_continuous_scale=["red","orange","yellowgreen","green"], size="temp", center={'lon':2.232999,"lat":46.327408})
fig.update_layout(title_x=0.5)
fig.show()